In [8]:
!git branch

* master


In [48]:
!git add .

In [49]:
!git commit -m "First data visualisation test"

[master cff16f4] First data visualisation test
 2 files changed, 91 insertions(+), 709 deletions(-)


In [50]:
!git push --set-upstream origin master

To https://github.com/Noorvh/SocComp-Term-Project.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to 'https://github.com/Noorvh/SocComp-Term-Project.git'
hint: Updates were rejected because the remote contains work that you do not
hint: have locally. This is usually caused by another repository pushing to
hint: the same ref. If you want to integrate the remote changes, use
hint: 'git pull' before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [9]:
import pandas as pd
import pickle 
import os

years = [1978, 1979, 1980, 1981, 1983, 1985, 1986, 1987, 1988, 1989, 1991, 1998, 2000]
df_years = []

In [10]:
for year in years:
    output_path = f"clean_data/{year}.csv"
    df = pd.read_csv(output_path)
    df_years.append(df)

In [11]:
print(len(df_years))

13


In [12]:
VARlist = ["VAR183", "VAR409", "VAR635", "VAR636", "VAR637", "VAR638", "VAR411", "VAR412", "VAR412", "VAR413", "VAR414"]

In [40]:
VARlist = ["VAR271"]

In [41]:
selected_dfs = []

# Loop through both the dataframe list and the corresponding years
for df, year in zip(df_years, years):

    # Find all columns matching any VAR code
    cols = [col for col in df.columns if any(var in col for var in VARlist)]

    # If relevant columns exist, extract and store them
    if cols:
        temp = df[cols].copy()
        temp["YEAR"] = year     # add year column
        selected_dfs.append(temp)

# Combine everything into one total dataframe
total_df = pd.concat(selected_dfs, axis=0, ignore_index=True)

total_df.head()


,VAR271,YEAR,VAR271 | INSPRAAK STUDENTEN OP BESTUUR UNIVERSITE | -6 = GEEN OPGAVE\n-3 = GEEN MENING\n1 = VEEL GROTER\n2 = BEETJE GROTER\n3 = ZO BLIJVEN\n4 = BEETJE MINDER\n5 = VEEL MINDER,VAR271 | INSPRAAK STUDENTEN OP BESTUUR UNIVERSITE | (1)VEEL GROTER (2)BEETJE GROTER (3)ZO BLIJVEN (4)BEETJE MINDER (5)VEEL MINDER (-3)GEEN MENING (-6)GEEN OPGAVE,VAR271 | (-6)GEEN OPGAVE (-3)GEEN MENING (1)VEEL GROTER (2)BEETJE GROTER (3)ZO BLIJVEN (4)BEETJE MINDER (5)VEEL MINDER,VAR271 | INSPRAAK STUDENTEN OP BESTUUR UNIVERSITE | -6 = GEEN OPGAVE\n-3 = GEEN MENING\n1 = VEEL GROTER\n2 = BEETJE GROTER\n3 = ZO BLIJVEN\n4 = BEETJE MINDER\n5 = VEEL MINDER\n21 = Apr 04 Page 34,VAR271 | 96 inspraak studenten op bestuur univ | -7 = uitgesloten\n-6 = geen opgave\n-3 = geen mening\n1 = veel groter\n2 = beetje groter\n3 = zo blijven\n4 = beetje minder\n5 = veel minder,VAR271 | 140 Inspraak studenten op bestuur van universiteit moet gr | -6 = geen opgave\n-3 = geen mening\n1 = veel groter\n2 = beetje groter\n3 = zo blijven\n4 = beetje minder\n5 = veel minder,"VAR271 | Inspraak studenten moet | -7 = Uitgesloten\n-6 = Geen opgave\n-5 = N.v.t.\n-3 = Geen mening\n-2 = Wil niet zeggen\n1 = Volkomen eens\n2 = Gr lijn eens\n3 = Noch, noch\n4 = Eigenl oneens\n5 = Helem oneens 16:07:28 Steinmetz Archive SUN SPARC Solaris 2.3","VAR271 | Inspraak studenten bestuur universi | -7 = Uitgesloten\n-3 = Geen mening\n1 = Volkomen eens\n2 = Gr lijn eens\n3 = Noch, noch\n4 = Eigenl oneens\n5 = Helem oneens 15:03:54 Steinmetz Archive SUN SPARC Solaris 2.3"
0,2.0,1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-6.0,1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3.0,1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are many questions with an answer option "weet niet" aka "I don't know". Do we remove these or not? For now we have removed.
And do we do categorical variables or do we do numerical variables - big choice!!

In [42]:
import pandas as pd
import re

def base_var(colname):
    """Extract base VAR code from a column name."""
    match = re.match(r'(VAR\d+)', colname)
    return match.group(1) if match else None


def merge_var_columns(df):
    """Merge columns sharing the same base VAR code."""
    merged = {}
    
    # Group columns by base code
    var_groups = {}
    for col in df.columns:
        base = base_var(col)
        if base:
            var_groups.setdefault(base, []).append(col)
        else:
            merged[col] = df[col]  # keep non-VAR columns (like YEAR)
    
    # Merge each VAR group
    for base, cols in var_groups.items():
        if len(cols) == 1:
            merged[base] = df[cols[0]]  # just rename to clean VAR code
        else:
            # Merge: take the first non-null value row-by-row
            merged[base] = df[cols].bfill(axis=1).iloc[:, 0]
    
    # Return as a new clean dataframe
    return pd.DataFrame(merged)


# --- APPLY THIS MERGING TO YOUR total_df ---

clean_total_df = merge_var_columns(total_df)


In [43]:
import pandas as pd

# 1. Select only VAR columns that are numeric
var_cols = [col for col in clean_total_df.columns if col.startswith("VAR")]
df_numeric = clean_total_df[["YEAR"] + var_cols].apply(pd.to_numeric, errors="coerce")

# 2. Remove unwanted values (-3, -6)
df_numeric[var_cols] = df_numeric[var_cols].replace([-1, -2, -3, -4, -5, -6, -7], pd.NA)
#print(df_numeric)

# 3. Compute the mean per year, ignoring NaN safely
yearly_means = df_numeric.groupby("YEAR").mean()
yearly_means = yearly_means.dropna(axis=1, how="all")  # drop columns all NaN
yearly_means.index = pd.to_numeric(yearly_means.index, errors='coerce')

# 4. Save CSV
yearly_means.reset_index().to_csv('clean_data/yearly_means.csv', index=False)
print(yearly_means)


        VAR271
YEAR          
1979  3.157559
1980  2.999341
1981  2.997678
1983  3.008203
1985  2.893487
1986   2.92678
1987  2.808702
1988  2.857234
1989  2.763537
1991  2.775465
1998  2.609375
2000  2.656923


In [39]:
print(clean_total_df)

      YEAR  VAR340
0     1980     4.0
1     1980     4.0
2     1980     1.0
3     1980     4.0
4     1980     3.0
...    ...     ...
6002  1985     4.0
6003  1985    -3.0
6004  1985     4.0
6005  1985     3.0
6006  1985     3.0

[6007 rows x 2 columns]


In [18]:
import plotly.graph_objects as go

# 5. Plot with Plotly
fig = go.Figure()

for col in yearly_means.columns:
    series = yearly_means[col].dropna()
    if series.empty:
        continue
    
    fig.add_trace(go.Scatter(
        x=series.index.astype(int),
        y=series.values,
        mode='lines+markers',
        name=col
    ))

# Layout
fig.update_layout(
    title="Average Response per VAR by Year",
    xaxis_title="Year",
    yaxis_title="Average Response",
    legend_title="Variables",
    width=900,
    height=600
)

fig.show()

ModuleNotFoundError: No module named 'plotly'

In [4]:
!pip install plotly

In [5]:
!conda install plotly

^C
